This script will make it so all images are normalized the same by taking the global min and max per band and applying it to each image.  Essentially I need to find across all training data and training bands what is the minimum and maximum values so all images are normalized the same.  I do this to avoid tiling appearences when deploying the model 

In [7]:
import pandas as pd
import numpy as np
import os

Function to get the the min-max globally but use the 5 and 95% cutoffs, I am not using this right now. 

In [ ]:
def minMax(x):
    return pd.Series(index=['min','max'],data=[x.percentile(5),x.percentile(95)])


Apply the minMax function by looping through all data.  Note we have so much data I sample only 30% of it per image. 

In [8]:
##path to training, validation and testing
train_files = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_proj_pos_0_128_training_files.csv')['Files'].tolist()
val_files = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_proj_pos_0_128_validation_files.csv')['Files'].tolist()
test_files = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_proj_pos_0_128_testing_files.csv')['Files'].tolist()



final = []


def get_min_max(pathways):
    
    
    for f in pathways:

        base = os.path.basename(f)
        
        try:
            x = np.load(f)
            x = x.astype(float)
            x[x == 0] = np.nan
            x[x == -999] = np.nan

            #convert to dataframe
            in_shape = x.shape

            #turn to dataframe to normalize
            x = x.reshape(x.shape[0] * x.shape[1], x.shape[2])

            # x = pd.DataFrame(x).dropna().sample(frac = 0.7)

            x = pd.DataFrame(x).dropna()
            # min_max = x.apply(minMax)

            final.append(x)
        except:
            
            print(f"{f} failed")
            
            
            os.remove(f)
        
    return()

get_min_max(train_files)
get_min_max(test_files)
get_min_max(val_files)

final4 = pd.concat(final).quantile([.01, .99])
final4
final4.reset_index().to_csv("/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_global_min_max_cutoff_proj_pos.csv", index = False)

()